# For this notebook please first download and extract the data and save the data in the directory specified in "data_location". Note that depending on how much data is in the location it might take very long and also need lots of RAM.

In [1]:
data_location = "selected_data"
save_to = "reduced_21_data"

In [1]:
#Imports
import os
from os.path import join
from datetime import datetime

import pandas as pd
import dask
import dask.dataframe as dd

In [1]:
%%time
import utils.preprocess as preprocess
from dask.distributed import progress

config = preprocess.load_default_config()
config["mem_lim"] = "6GB"
config["load_from"] = "parquet"
config['verbose'] = 2

_, delayed_save = preprocess.preprocess(config, root_dir="test", uncomp_dir = "test")


with preprocess.get_dask_compute_environment() as client:
        f = client.persist(delayed_save)
        progress(f)
        f.compute()

['test\\part-00000.csv', 'test\\part-00001.csv', 'test\\part-00002.csv', 'test\\part-00003.csv', 'test\\part-00004.csv', 'test\\part-00005.csv', 'test\\part-00006.csv', 'test\\part-00007.csv', 'test\\part-00008.csv', 'test\\part-00009.csv']
Outputting preprocessed files
Compute environment established: <Client: 'tcp://127.0.0.1:57084' processes=2 threads=4, memory=11.18 GiB>
Indexing finished
The uncompressed dataset is saved to the disk.


AbstractMethodError: This method must be defined in the concrete class Series

In [3]:
import pandas as pd
pd.read_parquet("test\\preprocessed\\part.1.parquet")

,bert_base_multilingual_cased_tokens,hashtags,tweet_id,medias,links,domains,type,language,timestamp,a_user_id,...,b_follower_count,b_following_count,b_is_verified,b_account_creation,a_follows_b,reply,retweet,retweet_comment,like,idx
__null_dask_index__,,,,,,,,,,,,,,,,,,,,,
0,101\t10485\t131\t177\t13009\t10261\t27874\t102...,None,C789139E04D060218D1FF0D3637FC51E,GIF,None,None,TopLevel,488B32D24BD4BB44172EB981C1BCA6FA,1614060231,AB77058178EEAC0A68B3EF17987121D8,...,54885,42664,False,1443867630,True,0,0,1614060648,0,258323
1,101\t56898\t137\t19482\t10317\t14526\t24387\t1...,None,EF9FC4ABFE2920E45CD0C8207FE6989C,None,None,None,Retweet,B0FA488F2911701DD8EC5B1EA5E322D8,1612726580,F530AC292F66F67C86BA10DB4AE13E70,...,619,605,False,1357779524,True,0,0,0,0,258324
2,101\t56898\t137\t10474\t85656\t10403\t40825\t1...,None,DAA4481C1696AABC16E5B5C830AB8B67,None,None,None,Retweet,488B32D24BD4BB44172EB981C1BCA6FA,1612752638,147B3B61778A2C5C55A1025BDC6A754E,...,469,432,False,1361999647,True,0,0,0,0,258325
3,101\t43843\t10741\t119\t18424\t20442\t11940\t1...,None,5B0BA86110D2BBBFD44C738D7D48592A,None,None,None,TopLevel,488B32D24BD4BB44172EB981C1BCA6FA,1612530302,F433C64C407673A25E58CF1F28A7FF78,...,364,258,False,1288726560,False,0,0,0,1612531253,258326
4,101\t137\t156\t10715\t10133\t11447\t22659\t234...,None,06CD067EA3EE659E053FB07EFC37C6E2,Photo,None,None,TopLevel,E6936751CBF4F921F7DE1AEF33A16ED0,1612432058,561ADEA7F5E34A24C95048CD4B86EC56,...,23,48,False,1595477877,True,0,0,0,1612644059,258327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258807,101\t12541\t10263\t112\t187\t28150\t136\t102,None,5280A8AB75D33A9AC9658C65B05151F5,None,None,None,TopLevel,99CA116BF6AA65D70F3C78BEBADC51F0,1613701571,3234DB25E2AC3A6930C269A919FFD179,...,245,214,False,1446183899,True,0,0,0,0,517130
258808,101\t100\t93495\t83822\t10238\t10131\t45707\t1...,None,F0AF5B737D7FA87E66F04A90ED04311C,Photo\tPhoto,None,None,TopLevel,E6936751CBF4F921F7DE1AEF33A16ED0,1613060615,2C34DFC23343A30095C15414A7CB8AC2,...,132,175,False,1522766411,False,0,0,0,1613068262,517131
258809,101\t67552\t11153\t81769\t11015\t79471\t10361\...,None,EC5D708867913F774FC3DAED64747082,None,None,None,Quote,1F73BB863A39DB62B4A55B7E558DB1E8,1612697872,2F3742902DAF5E66ED5248BBB5DAF777,...,308,625,False,1573853028,False,0,0,0,1612698287,517132


In [2]:
import dask.dataframe as dd
ddf = dd.read_parquet("test\\preprocessed", index="idx")
ddf.head()

,bert_base_multilingual_cased_tokens,hashtags,tweet_id,medias,links,domains,type,language,timestamp,a_user_id,...,b_user_id,b_follower_count,b_following_count,b_is_verified,b_account_creation,a_follows_b,reply,retweet,retweet_comment,like
idx,,,,,,,,,,,,,,,,,,,,,
1,101\t56898\t137\t10148\t96858\t18193\t11211\t1...,DF367A73CB84842E4A6080F624241703,C24EA5C3E19A388895D0FA05EA9E742F,Photo,None,None,Retweet,E7F038DE3EAD397AEC9193686C911677,1612712747,419BAC1CC63FF819E98CFC5EFF52DFDA,...,761EDF35532C3D3758DD01D33619D243,220,638,False,1464236635,False,0,0,0,1612721339
2,101\t22800\t11206\t117\t13451\t20517\t25136\t1...,None,2B41BDE7839DE3117D994FDFC9BDA6E1,None,None,None,TopLevel,488B32D24BD4BB44172EB981C1BCA6FA,1614002023,547810FE65A44B3828FFA4587D1CA0C8,...,AE8B92F8CFE9F92C2942DEBBFE4F0FCF,392,487,False,1362687553,True,0,0,0,0
3,101\t119\t119\t119\t45857\t85270\t10169\t77086...,None,BAF10FE3481E874ED104AC0950F18784,None,None,None,Quote,488B32D24BD4BB44172EB981C1BCA6FA,1612452111,E18710A5B40F9187026AA49A220857BC,...,ED5574665DB1478CA23F13647178D9BE,1306,5000,False,1244857625,True,0,0,0,1612457689
4,101\t160\t39187\t10237\t100\t11337\t100\t13028...,None,BCDD3DDDA04C2E7843B3734620EE0C38,Photo,None,None,TopLevel,488B32D24BD4BB44172EB981C1BCA6FA,1612980390,0CC43EBBB61E292102397B8B08A73DDD,...,585F08165162A5C10DE2081ECFACCFC0,90,91,False,1551356429,False,0,0,0,1612981097
5,101\t56898\t137\t40586\t10305\t53244\t22659\t1...,86458A61FFFA24A5624A9AD8AA2F0F52,43845DCDBC2A2CA15C3B18431F48F1F8,None,None,None,Retweet,488B32D24BD4BB44172EB981C1BCA6FA,1612834174,29766FB1458BC9BB67EB7E61CBAB957C,...,CAD64086EF823706DEF953E615DB6743,158,411,False,1417656869,False,0,0,0,0


In [2]:
ddf_ = ddf.set_index("idx", sorted=True, compute=True)